In [1]:
import sys
sys.path.append('../')

from function import prep_data, make_new_folder, plot_losses, save_input_args, shift_x, plot_norm_losses
from dataload import CELEBA 
from models import DAE, DIS_Z, LINEAR_SVM

In [2]:
import torch
from torch import optim

from torchvision.utils import make_grid, save_image
from torchvision import transforms, datasets

import argparse

from time import time

import os
from os.path import join

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

In [3]:
testDataset = CELEBA(root='/Users/aab01/Documents/Data', train=False, transform=transforms.ToTensor())
testLoader = torch.utils.data.DataLoader(testDataset, batch_size=100, shuffle=False)
print('Data loaders ready.')

Data loaders ready.


In [4]:
NIMAGES=1000 # THIS NEEDS TO MATCH THE DATALOADER PARTITION POINT!
NZ=200
FILTER_SIZE=64 #2 for testing
BATCH_SIZE = 100
IM_SIZE = 64
dae = DAE(imSize=IM_SIZE, fSize=FILTER_SIZE, nz=NZ, sigma=1.0)
# AWS:dae.load_params('/home/ubuntu/tutorials/PyTorch/TonisModels/Ex_58/')
dae.load_params('./TonisModels/Ex_60/')

loading params...


In [9]:
for i, data in enumerate(testLoader):
    print(i)
    x, y = prep_data(data, useCUDA=dae.useCUDA)
    z_enc = dae.encode(x)
    z_enc_npy = z_enc.data.numpy()

0
1
2
3
4
5
6
7
8
9


In [13]:
xcorr = dae.corrupt(x)
encCorr = dae.encode(xcorr)

In [14]:
## Compare histograms for enc, z_samples and encCorr
fig2 = plt.figure()
nEnc, bEnc, _ = plt.hist(z_enc.cpu().data.numpy().flatten(), 100, normed=True)
nEncCorr, bEncCorr, _ = plt.hist(encCorr.cpu().data.numpy().flatten(), 100, normed=True)
nNorm, bNorm, _ = plt.hist(dae.sample_z(10000).cpu().data.numpy().flatten(), 100, normed=True)

In [15]:
import seaborn as sns
sns.set(style="darkgrid")
sns.set_context("paper", font_scale=1.5,rc={"lines.linewidth": 2.5})
fig3 = plt.figure();

plt.plot(bEnc[1:], nEnc, label='Encoded data')

plt.plot(bEncCorr[1:], nEncCorr, label='Encoded, corrupted data')

plt.plot(bNorm[1:], nNorm, label='Prior')

plt.title('DAAE')
plt.xlabel('Value')
plt.ylabel('PDF')
plt.legend()
plt.show()
plt.savefig('HisEnc-60.svg')